# Predicting NIFTY with Transformer

The following is a Pytorch implementation of a transformer model to predict NIFTY indices.

## Imports

In [ ]:
import os
import copy
from pathlib import Path
import warnings

import numpy as np
import pandas as pd
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor
from pytorch_lightning.loggers import TensorBoardLogger
import torch

from pytorch_forecasting import Baseline, TemporalFusionTransformer, TimeSeriesDataSet
from pytorch_forecasting.data import GroupNormalizer
from pytorch_forecasting.metrics import SMAPE, PoissonLoss, QuantileLoss
from pytorch_forecasting.models.temporal_fusion_transformer.tuning import optimize_hyperparameters

## Load Data

In [ ]:
data = pd.read_excel('NIFTY50.xlsx', sheet_name=None)

def copy_dict_without_keys(d, keys):
    return {x: d[x] for x in d if x not in keys}

input_data = copy_dict_without_keys(data, ['Companies List'])
output_data = copy_dict_without_keys(data, list(set(data.keys()) - {'in'}))

# prepare input data before merge
for sheet in input_data:
    # 1. replace '##...' by actual dates
    # ('##...' are placeholders for wider text)

    # 2. impute null values 
    input_data[sheet] = data[sheet].dropna(subset=['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'])

    # 3. Add a categorical column for company symbol
    input_data[sheet]['Symbol'] = sheet

    # 4. convert date to standard format
    input_data[sheet]['Date'] = pd.to_datetime(input_data[sheet]['Date'], dayfirst=True)

# impute null values in output data and convert date to standard format
output_data = output_data.dropna(subset=['Open', 'High', 'Low', 'Close', 'Adj Close'])
output_data['Date'] = pd.to_datetime(output_data['Date'], dayfirst=True)

for sheet in input_data:
    output_data[sheet] = (output_data['Date'] in input_data[sheet]['Date'])

# concatenate dataframes
input_data = pd.concat(input_data)

## Create Dataset and dataloaders

Convert the dataframe of raw data into a PyTorch Forecasting ``TimeSeriesDataSet``. 

## Transformer

Initialize and train a time-series forecasting transformer.

### Optimize learning rate

The optimal learning rate is identified using PyTorch Lightning learning rate finder.

For the TemporalFusionTransformer, the optimal learning rate seems to be slightly lower than the suggested one. Further, we do not directly want to use the suggested learning rate because PyTorch Lightning sometimes can get confused by the noise at lower learning rates and suggests rates far too low. Manual control is essential.

## Train Model